In [20]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm

In [27]:
path = Path('.')
dataframes = {}
columns =  ['dates','subframe_n','subframe_ind','rnti','direction','mcs']
for p in list(path.glob('new_A')):
    for file in tqdm(list(p.glob('*.csv*'))):
        df = pd.read_csv(file, names = columns)
        dataframes[file.parts[-1]]=df
    

100%|██████████| 42/42 [00:31<00:00,  1.31it/s]


In [28]:
dataframes['file_A.csv_24']

,dates,subframe_n,subframe_ind,rnti,direction,mcs
0,20170327021517,15,3.0,4736.0,1.0,1.0
1,20170327021517,65,5.0,15247.0,1.0,14.0
2,20170327021518,0,5.0,5844.0,1.0,18.0
3,20170327021518,17,3.0,28020.0,1.0,8.0
4,20170327021518,20,5.0,57850.0,1.0,12.0
5,20170327021519,94,5.0,9676.0,1.0,27.0
6,20170327021521,49,4.0,2710.0,1.0,11.0
7,20170327021521,84,5.0,63632.0,1.0,27.0
8,20170327021522,32,3.0,64184.0,1.0,15.0
9,20170327021523,66,4.0,38579.0,1.0,26.0
